# Master code for the Wikipedia data

### Mariana, Chichi, Nate, and Taylor

In [1]:
# Import statements
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import numpy as np
from datetime import datetime as dt


## Part 1: Getting data about every episode from seasons 9-18

#### Defining global variables

In [2]:
# Defining some primary variables

# Seasons we are interested in
seasons = list(range(9, 19))

# URL to Wikipedia page of the seasons and episodes
seasons_url = 'https://en.wikipedia.org/wiki/List_of_The_Bachelorette_(American_TV_series)_episodes'

# Base URL for Wikipedia pages of every season – to be formatted later
url_base = "https://en.wikipedia.org/wiki/The_Bachelorette_(American_season_%i)"

#### Getting every table from the Wikipedia page with information about every season

In [3]:
# Getting all the tables from the wikipedia page
page = rq.get(seasons_url)

# accessing page contents
soup = BeautifulSoup(page.content)

# parsing the contents as html
soup = BeautifulSoup(page.text, 'html.parser')

# finding all the wikitables
tables =soup.find_all('table', {'class':"wikitable"})

# Saving all the tables as dataframes to a list
all_tables = []
for table in tables:
    # reading the request as an html document
    df = pd.read_html(str(table))
    
    # saving it as a dataframe
    df = pd.DataFrame(df[0])
    
    # appending said dataframe to the list
    all_tables.append(df)


#### Formatting the tables as needed

In [4]:
# Formatting the tables as we need them to be:

# removing a double-layer column labels
all_tables[0].columns = [col[1] for col in all_tables[0].columns]

# setting the indices of the dataframe to the season number
all_tables[0].index = all_tables[0]["Season.1"]

# Dropping redundant columns caused by two-layer column labels
all_tables[0].drop(columns=["Season", "Season.1", "Episodes.1"],inplace=True)

# removing weird last table
all_tables.pop(-1)


# checking we are right
all_tables[18]

,No.overall,No. inseason,Title,Original air date [156],Prod.code [156],U.S. viewers(millions),Rating(18–49)
0,193,1,"""Week 1: Season Premiere""","October 19, 2021",1801.0,3.00[157],0.8/4[157]
1,194,2,"""Week 2""","October 26, 2021",1802.0,2.87[158],0.7/3[158]
2,195,3,"""Week 3""","November 2, 2021",1803.0,2.74[159],0.7/3[159]
3,196,4,"""Week 4""","November 9, 2021",1804.0,2.60[160],0.6/4[160]
4,197,5,"""Week 5""","November 16, 2021",1805.0,2.98[161],0.7[161]
5,198,6,"""Week 6""","November 23, 2021",1806.0,2.71[162],0.6/3[162]
6,199,7,"""Week 7: Hometown Dates""","November 30, 2021",1807.0,3.07[163],0.6/7[163]
7,200,8,"""The Men Tell All""","December 6, 2021",NaN,TBD,TBA
8,201,9,"""Week 8: Fantasy Suites""","December 14, 2021",1808.0,TBD,TBA
9,202,10,"""Week 9: Finale""","December 21, 2021",1809.0,TBD,TBA


#### Saving the tables in a dictionary keyed by season number

In [5]:
# now we want to have the tables saved in a dictionary with the key being the season number
seasons_table = defaultdict()

for i in range(9, 19):
    seasons_table[str(i)] = all_tables[i]


# Dictionary keyed by the season number with all episodes as dataframes
# Only includes seasons 9-18

# normalizing the column names for easier cleaning
for table in seasons_table.keys():
    seasons_table[table].columns = ["Episode Number", "In-Season Episode", "Title", "Original Air Date", "Prod Code", "Viewers", "Rating"]


#### Only extracing the relevant data (i..e Season episode number, title, air date)


In [6]:
# Now we want to extract only the data we want
season_episodes_dates = []

# Going through each table which referrs to each season
for season in seasons_table.keys():
    
    # and then for each episode in the season
    for episode in range(len(seasons_table[season])):
        
        # save the airing date as a variable to be used to make it a datetime object
        date = seasons_table[season]["Original Air Date"].iloc[episode]
        
        # removing weird nuances in the wikipedia page (from citations)
        if date[-1] == "]":
            date = date[0:-3]
        
        # saving that date (encoded as unicode) as an ascii encoded datetime object
        date_time = dt.strptime(date.replace("\xa0", " "), "%B %d, %Y")
        
        # saving the data to a list in the format of:
        # [Season number, episode number, and unified datetime]
        season_episodes_dates.append([season, seasons_table[season]["In-Season Episode"].iloc[episode], seasons_table[season]["Title"].iloc[episode], str(date_time)[:10]])
              

### Now we have the data formatted as
    [Season #, Episode Number, Title of Episode, Date of Airing]

#### Now we want to save this data as a DataFrame for easier manipulation in Part 2

In [7]:
# Creating a dataframe with the data and normalizing the columns
s_e_d = pd.DataFrame(season_episodes_dates)
s_e_d.columns = ["Season", "Episode", "Title", "Date Aired"]

# Making the episode titles normalized
for i in range(len(s_e_d)):
    s_e_d["Title"].iloc[i] = s_e_d["Title"].iloc[i].split(":")[0].replace("\"", "")
    
s_e_d.head(20)

/Users/taylor/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Season,Episode,Title,Date Aired
0,9,1,Week 1,2013-05-27
1,9,2,Week 2,2013-06-03
2,9,3,Week 3,2013-06-10
3,9,4,Week 4,2013-06-17
4,9,5,Week 5,2013-06-24
5,9,6,Week 6,2013-07-01
6,9,7,Week 7,2013-07-08
7,9,8,Week 8,2013-07-15
8,9,9,The Men Tell All,2013-07-22
9,9,10,Week 9,2013-07-29


#### Saving this dataframe as a csv to do some manual cleanup of two-part episodes and finnicky edge cases

In [8]:
s_e_d.to_csv("./season_data.csv")

## Part 2: Contestant Data

#### Defining a function to derive the useful information from each season's wikipedia page in the form of 
    [Name, Episode Voted Off]

In [9]:
def get_names_outcomes(season):
    url = url_base % season
    
    # Getting all the tables from the wikipedia page
    page = rq.get(url)

    # accessing page contents
    soup = BeautifulSoup(page.content)

    # parsing the contents as html
    soup = BeautifulSoup(page.text, 'html.parser')

    # finding all the wikitables
    tables =soup.find_all('table', {'class':"wikitable"})

    # get the first table on the page
    contestants = tables[0]

    # reading the request as an html document
    df = pd.read_html(str(contestants))

    # saving it as a dataframe
    df = pd.DataFrame(df[0])

    # isolating the columns we want
    names = df["Name"]
    outcome = df["Outcome"].values
    names_clean = []

    # stupid cleaning of the names because BS doesnt take out references!
    for name in names:
        names_clean.append(name.split("[")[0])

    # getting the data ready for the dataframe
    data = {"Name": names_clean,
            "Voted Off": outcome}

    # creating and returing the dataframe
    to_return = pd.DataFrame(data=data, columns=["Name", "Voted Off"])

    return to_return

In [10]:
tables_of_seasons = defaultdict()

# Getting contestants for all seasons
for season in seasons:
    tables_of_seasons[season] = get_names_outcomes(season)


## Part 3: Combining the DataFrames

#### Defining global variables and renaming dataframes

In [11]:
# Global list of season numbers
seasons = list(range(9, 19))

# renaming s_e_d
# Type: DataFrame
season_data = pd.read_csv("./season_data.csv")

# renaming the tables_of_seasons
# Type: Dictionary of dataframes, keyed by season number
contestant_data = tables_of_seasons

#### Iterate through the seasons and extract contestant information n times, where n is the number of episodes in that season

#### We want our initial table to look like this for now

|Season|Episode|Title|Date Aired|Contestants|Voted Off|
|--|--|--|--|--|--|
|Type: int|Type: int|Type: str|Type: datetime|Type: list|Type: NA|


#### Fixing the weird indexing column in the imported dataframe

In [12]:
season_data.drop(columns=["Unnamed: 0"], inplace=True)
season_data

,Season,Episode,Title,Date Aired
0,9,1,Week 1,2013-05-27
1,9,2,Week 2,2013-06-03
2,9,3,Week 3,2013-06-10
3,9,4,Week 4,2013-06-17
4,9,5,Week 5,2013-06-24
...,...,...,...,...
114,18,7,Week 7,2021-11-30
115,18,8,The Men Tell All,2021-12-06
116,18,9,Week 8,2021-12-14
117,18,10,Week 9,2021-12-21


#### Creating a list of lists with the same dimensions of the `season_data` variable. Each list will contain all the contestants from that season. 

In [13]:
df_of_lists = []

for season in seasons:
    contestants = contestant_data[season]["Name"].values
    for i in range(len(season_data[season_data["Season"] == season])):
        df_of_lists.append(contestants)
        
df_of_lists

[array(['Chris Siegfried', 'Drew Kenney', 'Brooks Forester', 'Zak Waddell',
        'Michael Garofola', 'James Case', 'Juan Pablo Galavis',
        'Kasey Stewart', 'Mikey Tenerelli', 'Ben Scott', 'Bryden Vukasin',
        'Zack Kalter', 'Brad McKinzie', 'Brandon Andreen', 'Dan Cox',
        'Brian Jarosinski', 'Nick Mucci', 'Robert Graham', 'Will Reese',
        'Diogo Custodio', 'Larry Burchett', 'Micah Heisler', 'Mike R.',
        'Nick Roy', 'Jonathan Vollinger'], dtype=object),
 array(['Chris Siegfried', 'Drew Kenney', 'Brooks Forester', 'Zak Waddell',
        'Michael Garofola', 'James Case', 'Juan Pablo Galavis',
        'Kasey Stewart', 'Mikey Tenerelli', 'Ben Scott', 'Bryden Vukasin',
        'Zack Kalter', 'Brad McKinzie', 'Brandon Andreen', 'Dan Cox',
        'Brian Jarosinski', 'Nick Mucci', 'Robert Graham', 'Will Reese',
        'Diogo Custodio', 'Larry Burchett', 'Micah Heisler', 'Mike R.',
        'Nick Roy', 'Jonathan Vollinger'], dtype=object),
 array(['Chris Siegfried

#### Making that list into a DataFrame

In [14]:
vec_contestants = pd.DataFrame(data={"Contestants": df_of_lists})
vec_contestants

,Contestants
0,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
1,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
2,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
3,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
4,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
...,...
114,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."
115,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."
116,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."
117,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."


#### Adding that vector to the `season_data` matrix, and rename to `master_data`. 

In [15]:
master_data = season_data.join(vec_contestants)
master_data

,Season,Episode,Title,Date Aired,Contestants
0,9,1,Week 1,2013-05-27,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
1,9,2,Week 2,2013-06-03,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
2,9,3,Week 3,2013-06-10,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
3,9,4,Week 4,2013-06-17,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
4,9,5,Week 5,2013-06-24,"[Chris Siegfried, Drew Kenney, Brooks Forester..."
...,...,...,...,...,...
114,18,7,Week 7,2021-11-30,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."
115,18,8,The Men Tell All,2021-12-06,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."
116,18,9,Week 8,2021-12-14,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."
117,18,10,Week 9,2021-12-21,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro..."


#### Now we create a column to contain a list of who was voted off in that episode

In [16]:
voted_off = pd.DataFrame(columns = ["Voted Off"])
master_data = master_data.join(voted_off)
master_data

,Season,Episode,Title,Date Aired,Contestants,Voted Off
0,9,1,Week 1,2013-05-27,"[Chris Siegfried, Drew Kenney, Brooks Forester...",NaN
1,9,2,Week 2,2013-06-03,"[Chris Siegfried, Drew Kenney, Brooks Forester...",NaN
2,9,3,Week 3,2013-06-10,"[Chris Siegfried, Drew Kenney, Brooks Forester...",NaN
3,9,4,Week 4,2013-06-17,"[Chris Siegfried, Drew Kenney, Brooks Forester...",NaN
4,9,5,Week 5,2013-06-24,"[Chris Siegfried, Drew Kenney, Brooks Forester...",NaN
...,...,...,...,...,...,...
114,18,7,Week 7,2021-11-30,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro...",NaN
115,18,8,The Men Tell All,2021-12-06,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro...",NaN
116,18,9,Week 8,2021-12-14,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro...",NaN
117,18,10,Week 9,2021-12-21,"[Brandon Jones, Joe Coleman, Nayte Olukoya, Ro...",NaN


#### We have gone through the data in `master_data` after saving it to a CSV to perform some manual cleaning of episodes where no one gets voted off. Here, we pull back in the CSV as a DataFrame

In [17]:
master_data_clean = pd.read_csv("./master_data_clean.csv")
master_data_clean.drop(columns=["Unnamed: 0"], inplace=True)
master_data_clean

,Season,Episode,Title,Date Aired,Contestants,Voted Off
0,9,1,Week 1,5/27/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,NaN
1,9,2,Week 2,6/3/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,NaN
2,9,3,Week 3,6/10/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,NaN
3,9,4,Week 4,6/17/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,NaN
4,9,5,Week 5,6/24/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,NaN
...,...,...,...,...,...,...
95,18,4,Week 4,11/9/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,NaN
96,18,5,Week 5,11/16/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,NaN
97,18,6,Week 6,11/23/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,NaN
98,18,7,Week 7,11/30/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,NaN


#### Here we are sequentially creating a list of lists where each list represents which contestants were voted off during that episode.

In [18]:
# Global list of length 100 (to match master_data_clean)
all_voted_off = []

for season in seasons:
    
    # Getting the contestant data from each season
    comp_conts = contestant_data[season]
    # only looking at one season at a time from master_data_clean
    master_block = master_data_clean[master_data_clean["Season"] == season]

    # initializing a temporary list to be added to the global list
    season_voted_off = []
    
    # running through each episode in the season
    for episode in range(len(master_block)):

        # define the week
        week = master_block.iat[episode, 2]

        # initialize a temporary list of contestants to be added to the season list
        voted_off = []

        # for every contestant that season, see if their outcome (i.e. when they were voted off) matches the week
        # if so, add them to the voted off list for that week
        for contestant in range(len(comp_conts)):
            if comp_conts.iat[contestant, 1] == week:
                voted_off.append(comp_conts.iat[contestant, 0])
                
        # add that list to the season list of people voted off
        season_voted_off.append(voted_off)
        
    # we didnt want a list of lists of lists, so this simply breaks it down into a list of lists
    for vo in season_voted_off:
        all_voted_off.append(vo)


#### Checking to see if what we have is what we want, which is a list containing lists of every character voted off in each episode

In [19]:
all_voted_off

[['Diogo Custodio',
  'Larry Burchett',
  'Micah Heisler',
  'Mike R.',
  'Nick Roy',
  'Jonathan Vollinger'],
 ['Nick Mucci', 'Robert Graham', 'Will Reese'],
 ['Brandon Andreen', 'Dan Cox', 'Brian Jarosinski'],
 ['Zack Kalter', 'Brad McKinzie'],
 ['Mikey Tenerelli', 'Ben Scott', 'Bryden Vukasin'],
 ['James Case', 'Juan Pablo Galavis', 'Kasey Stewart'],
 ['Michael Garofola'],
 ['Zak Waddell'],
 ['Brooks Forester'],
 [],
 ['Emil Schaffroth',
  'Jason Leep',
  'Josh Bauer',
  'Mike Campanelli',
  'Rudie Dane',
  'Steven Woolworth'],
 ['Carl King', 'Craig Muhlbauer', 'Nick Sutter'],
 ['Bradley Wisk', 'Brett Melnick', 'Ron Worrell'],
 ['Tasos Hernandez', 'Eric Hill'],
 ['Andrew Poole', 'Marquel Martin', 'Patrick Jagodzinski'],
 ['Joseph John "J.J." O\'Brien', 'Cody Sattler'],
 ['Brian Osborne', 'Dylan Petitt'],
 ['Marcus Grodd'],
 ['Chris Soules'],
 [],
 ['Bradley Cox',
  'David',
  'Josh Seiter',
  'Shawn Evans',
  'Brady Toops',
  'Ryan McDill'],
 ['Cory Shivar', 'Daniel Finney', 'Kupah 

#### Now we need to add that vector to the column for the `master_data`!

In [20]:
# removing placeholder Voted Off column
master_data_clean.drop(columns=["Voted Off"], inplace=True)

In [21]:
# making our data into a dataframe for easy joining
avo = pd.DataFrame(data={"Voted Off": all_voted_off})

# joining
master_data_final = master_data_clean.join(avo)

In [22]:
master_data_final

,Season,Episode,Title,Date Aired,Contestants,Voted Off
0,9,1,Week 1,5/27/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,"[Diogo Custodio, Larry Burchett, Micah Heisler..."
1,9,2,Week 2,6/3/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,"[Nick Mucci, Robert Graham, Will Reese]"
2,9,3,Week 3,6/10/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,"[Brandon Andreen, Dan Cox, Brian Jarosinski]"
3,9,4,Week 4,6/17/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,"[Zack Kalter, Brad McKinzie]"
4,9,5,Week 5,6/24/13,['Chris Siegfried' 'Drew Kenney' 'Brooks Fores...,"[Mikey Tenerelli, Ben Scott, Bryden Vukasin]"
...,...,...,...,...,...,...
95,18,4,Week 4,11/9/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,"[Chris Gallant, Romeo Alexander, Will Urena]"
96,18,5,Week 5,11/16/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,"[Casey Woods, Leroy Arthur, Chris Sutton]"
97,18,6,Week 6,11/23/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,"[Olumide ""Olu"" Onajide, Rick Leach, Martin Gel..."
98,18,7,Week 7,11/30/21,['Brandon Jones' 'Joe Coleman' 'Nayte Olukoya'...,[Rodney Matthews]


#### Finally, saving that to a CSV for sharing among teammates

In [23]:
master_data_final.to_csv("./wikipedia_master.csv")